In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from pandas_profiling import ProfileReport
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [63]:
cols = ["YEAR", "DAY", "HOUR", "ID for IMF", "ID for SW Plasma", "# of points in IMF averages",
         "# of points in Plasma averages", "BX", "BY", "BZ", "SW Plasma Temperature, K", 
         "SW Proton Density, N/cm^3", "SW Plasma Speed, km/s","SW Plasma flow long. angle", 
         "SW Plasma flow lat. angle", "Flow pressure", "Kp index", "Dst-index", "AL-index", "AU-index"]
train = ''
target = ''
train_cols = []
target_cols = []

In [ ]:
report = ProfileReport(df)
report.to_file('Assurance.html')

In [ ]:
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer(n_neighbors=5, weights="uniform")
train = knn_imputer.fit_transform(train)
train = train[train_cols]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train = scaler.fit_transform(train)
target = scaler.fit_transform(target)

In [ ]:
n_input = 12
generator = TimeseriesGenerator(train, target, length=n_input, batch_size=1)


model = Sequential()
model.add(LSTM(1001, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [64]:
from sklearn.model_selection import KFold
k = 10
kf = KFold(n_splits=k,random_state=2020,shuffle=True)  ; 
for i , (train_indices, test_indices) in enumerate(kf.split(train)):
    train_train_, y_train_ = train.iloc[train_indices], target[train_indices]
    train_valid_, y_valid_ = train.iloc[test_indices], target[test_indices]
    
    
    model.fit(train_train_, y_train_)

    pred = model.predict(train_valid_)
    oof[test_indices] = pred
    print ('Fold',i,'- score:',mean_squared_error(y_valid_, pred))
    
    pred_test += model.predict(train_test) / k ;
    
print (mean_squared_error(Y, oof))